In [1]:
import pandas as pd

In [123]:
df=pd.read_csv('dataset/ronb_dataset.csv',delimiter=';')

In [124]:
df.head()

,Description,Intent
0,Flood/Landslide Update: Death count due to cur...,Sad
1,Kathmandu bata Pokhara jadai gareko Car Trisul...,Sad
2,Growing danger in Kathmandu area: A boy of aro...,Sad
3,Kathmandu District Administration Office le ma...,Surprise
4,Popular Netflix Series Money Heist ( La Casa d...,Surprise


In [125]:
df.shape

(625, 2)

In [126]:
df.tail()

,Description,Intent
620,Medical Students protesting in front of Kathma...,Sad
621,CIAA (Akhtiyar) le Krishi Samagri Company Ltd ...,Surprise
622,Swikar Poudel from Pokhara got 3rd position in...,Surprise
623,Current fiscal year ko first 5 months ma Nepal...,Sad
624,Chauchau ko Carton bhitra NRs. 50 Lakh Cash lu...,Funny


In [127]:
df.columns

Index(['Description', 'Intent'], dtype='object')

In [128]:
df['Intent'].value_counts()

Surprise    280
Sad         241
Funny        72
Angry        32
Name: Intent, dtype: int64

In [129]:
df['Intent'].unique()

array(['Sad', 'Surprise', 'Funny', 'Angry'], dtype=object)

In [130]:
df[df['Intent']=='nan']

,Description,Intent


In [131]:
df[df['Intent'].isna()]

,Description,Intent


In [132]:
df.iloc[14].values

array(["A discussion about Nepal's current situation happened at UK's parliament where the members talked about helping Nepal in present condition. They also talk about how Nepal's Gurkhas have helped UK in need. P.S. UK is sending medical team next week to help Nepal.",
       'Surprise'], dtype=object)